<a href="https://colab.research.google.com/github/custom-hyper/CoinGeckoAPI/blob/main/Screener_%E2%80%93_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Screening Criteria

Change Requests:
* Remove zeros of market capitalization
* Combine with other tables that don't often update but are useful for research

In [1]:
#@title Connect to FNS { display-mode: "code" }
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
#@title pip  { display-mode: "code" }
!pip install plotly
!pip install pytrends
!pip install pycoingecko
!pip install pandas
!pip install tweepy
!pip install requests
!pip install pipreqs
!pip install configparser
!pip install psycopg2-binary==2.8.6



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pytrends: filename=pytrends-4.8.0-py3-none-any.whl size=16126 sha256=befdd6d6510292d593118c794e4fb40d5cfd79ba743f49e5b3ab172f61d7b84a
  Stored in directory: /root/.cache/pip/wheels/07/6f/5c/8174f98dec1bfbc7d5da4092854afcbcff4b26c3d9b66b5183
Successfully built pytrends
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes

In [3]:
#@title Ta-lib – Extra steps { display-mode: "code" }
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/libta-lib0_0.4.0-oneiric1_amd64.deb -qO libta.deb
!wget https://launchpad.net/~mario-mariomedina/+archive/ubuntu/talib/+files/ta-lib0-dev_0.4.0-oneiric1_amd64.deb -qO ta.deb
!dpkg -i libta.deb ta.deb
!pip install ta-lib

Selecting previously unselected package libta-lib0.
(Reading database ... 155639 files and directories currently installed.)
Preparing to unpack libta.deb ...
Unpacking libta-lib0 (0.4.0-oneiric1) ...
Selecting previously unselected package ta-lib0-dev.
Preparing to unpack ta.deb ...
Unpacking ta-lib0-dev (0.4.0-oneiric1) ...
Setting up libta-lib0 (0.4.0-oneiric1) ...
Setting up ta-lib0-dev (0.4.0-oneiric1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.3) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 269 kB 23.7 MB/s 
  Created wheel for ta-lib: filename=TA_Lib-0.4.24-cp37-cp37m-linux_x86_64.whl size=1465737 sha256=1c581b7b0e5addaf29c6d6e80d7cd9148eb6e6f5a190eb23a187c2983d8c8e78
  Stored in directory: /root/.cache/pip/whe

In [4]:
#@title Import Project  Dependencies { display-mode: "code" }
import requests
from datetime import datetime as dt
from datetime import timedelta
import talib as ta
import time
import json
import seaborn as sns
from google.colab import data_table
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
import logging
import sys
from tqdm.notebook import tqdm
import os
import sqlite3 as sq
import psycopg2 as pg
import configparser
from sqlalchemy import create_engine
from pycoingecko import CoinGeckoAPI
import numpy as np
from __future__ import division

## Setup SQLite


In [5]:
#@title  Connect { display-mode: "code" }
path = '/content/drive/MyDrive/workspace/pipelines/db/' #@param {type: 'string'
database = "db.sqlite" #@param {type: 'string'
conn_SQLite = sq.connect(path+database)

In [6]:
#@title   Show Available Tables { display-mode: "code" }
cursor = conn_SQLite.cursor()

%time
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
#cursor.execute("DROP TABLE 'proprietary_Twitter_Partnership_ohlc';")
cursor.fetchall()

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.34 µs


[('deFi_lama_total_TVL',),
 ('proprietary_Twitter_Partnership_ohlc',),
 ('proprietary_master_id',),
 ('CoinGecko_ohlc',),
 ('proprietary_tweets_extract',),
 ('proprietary_Twitter_partnerships_announcements',),
 ('proprietary_airtable_watch_all',),
 ('proprietary_dashboard',),
 ('watchlist_CoinGecko_ohlc',),
 ('proprietary_Twitter_id',),
 ('deFi_lama_tvl',),
 ('testing',),
 ('CoinGecko_main',),
 ('deFi_lama_general',),
 ('CoinGecko_derivatives',),
 ('proprietary_screener',),
 ('CoinGecko_currencies',),
 ('CoinGecko_global',),
 ('CoinGecko_asset_platforms',)]

 # Load Partnership Dataset

[Twitter API](https://developer.twitter.com/en/docs/twitter-ads-api/timezones): Datetime values are always returned in UTC time (as indicated by the Z at the end of the datetime value.) 

# Extract Data

Data source: [CoinGecko API wrapper](https://github.com/man-c/pycoingecko)

In [7]:
cg = CoinGeckoAPI()
output = cg.get_derivatives(include_tickers ='unexpired')
data = pd.json_normalize(output,max_level=4)
data = pd.DataFrame(output, columns =(
    ['market', 'symbol', 'index_id', 'price', 'price_percentage_change_24h',
       'contract_type', 'index', 'basis', 'spread', 'funding_rate',
       'open_interest', 'volume_24h', 'last_traded_at', 'last_traded_at_f']))

#@title Get Derivatives { display-mode: "code" }
data['last_traded_at_f'] = pd.to_datetime(data['last_traded_at'], unit='s')
CoinGecko_derivatives = "CoinGecko_derivatives"

data.to_sql(CoinGecko_derivatives, conn_SQLite, if_exists='replace', index=False) # writes to file
conn_SQLite.commit()

# Just be sure any changes have been committed or they will be lost.

CoinGecko_derivatives = pd.read_sql("""
select * 
from {} 

""".format(CoinGecko_derivatives), conn_SQLite)
CoinGecko_derivatives




,market,symbol,index_id,price,price_percentage_change_24h,contract_type,index,basis,spread,funding_rate,open_interest,volume_24h,last_traded_at,last_traded_at_f
0,Binance (Futures),BTCUSDT,BTC,20921.23,3.264329,perpetual,20938.163979,0.058129,0.01,0.010000,2.348922e+09,1.236015e+10,1657217360,2022-07-07 18:09:20
1,Bitget Futures,BTCUSDT_UMCBL,BTC,20936.16,3.422983,perpetual,20937.959309,-0.002582,0.01,-0.006800,1.971912e+09,4.914379e+09,1657217519,2022-07-07 18:11:59
2,FTX (Derivatives),BTC-PERP,BTC,20921.0,3.477100,perpetual,20927.705000,0.032049,0.01,-0.014400,1.360255e+09,2.171068e+09,1657217453,2022-07-07 18:10:53
3,Binance (Futures),ETHUSDT,ETH,1224.5,7.078973,perpetual,1225.204416,0.034652,0.01,0.007128,1.226357e+09,7.135176e+09,1657217360,2022-07-07 18:09:20
4,MEXC Global (Futures),BTC_USDT,BTC,20926.16,3.391463,perpetual,20938.100000,0.045870,0.01,0.005000,9.773849e+08,1.475068e+09,1657217515,2022-07-07 18:11:55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3093,FTX (Derivatives),GLD-0930,GLD,162.73,0.323674,futures,162.300000,-0.261177,0.11,0.000000,NaN,4.874250e+00,1657208479,2022-07-07 15:41:19
3094,FTX (Derivatives),CGC-0930,CGC,2.68,0.000000,futures,2.737500,2.336449,0.90,0.000000,1.393434e+04,8.290580e+02,1657135073,2022-07-06 19:17:53
3095,FTX (Derivatives),BNTX-0930,BNTX,159.77,-0.646726,futures,167.240000,4.675471,0.16,0.000000,5.176550e+03,7.127050e+02,1657140786,2022-07-06 20:53:06
3096,FTX (Derivatives),GDXJ-0930,GDXJ,30.33,0.000000,futures,31.275000,3.115727,0.22,0.000000,NaN,1.213200e+03,1657156712,2022-07-07 01:18:32


In [ ]:
frames = []
cg = CoinGeckoAPI()

try:

  start = 0
  ends = 135

  for i in tqdm(range(start,ends)):

    output = cg.get_coins_markets(vs_currency='usd', page = i)
    
    data = pd.json_normalize(output,max_level=4)
    frames.append(data)
    time.sleep(60/50)

except ConnectionError:
    time.sleep(60)

full_table = pd.concat(frames) #Careful, this line must be outside of the for loop



full_table['price_range_percent_24h'] =  (full_table['high_24h'] - full_table['low_24h'] ) / 0.5*(full_table['high_24h'] + full_table['low_24h'] ) 

try:
  circ_cap = (full_table['current_price'] * full_table['circulating_supply'])
  full_table['circ_cap_rotation_24h'] = full_table['total_volume'] / circ_cap
except Exception:
  pass

try:
  full_table['float_rotation_24h'] = full_table['total_volume'] / full_table['market_cap']
except Exception:
  pass 
try:
  full_table['float_rotation_24h_rank']  =  full_table['float_rotation_24h'].rank(pct=True)
except Exception:
  np.nan 

#full_table.replace([np.inf, -np.inf], np.nan, iplace=True)

full_table['circulating_cap'] = full_table['current_price'] * full_table['circulating_supply']
full_table['total_cap'] = full_table['current_price'] * full_table['total_supply']
full_table['max_cap'] = full_table['current_price'] * full_table['max_supply']

full_table['price_change_percentage_24h'] = full_table['price_change_percentage_24h']/100
full_table['ath_change_percentage'] = full_table['ath_change_percentage']/100
full_table['market_cap_change_percentage_24h'] = full_table['market_cap_change_percentage_24h']/100
full_table['atl_change_percentage'] = full_table['atl_change_percentage']/100


full_table['dilution_change_percentage_24h'] = full_table['market_cap_change_percentage_24h'] - full_table['price_change_percentage_24h']
full_table['prop_timestamp'] = int( time.time_ns() / 1000 )
full_table['prop_timestamp'] = full_table['prop_timestamp'].astype(np.int8)

"""
full_table['dilution_ratio'] = full_table['dilution_ratio'].dropna()

full_table['dilution_ratio_rank_pct'] = full_table['dilution_ratio'].rank(pct=True)
"""
full_table['derivatives'] = 'yes' if full_table['symbol'].values == CoinGecko_derivatives['index_id'].values else 'no'

full_table['CoinGecko_link'] = str('https://www.coingecko.com/en/coins/') + full_table['id']

full_table['symbol'] = full_table['symbol'].str.upper()


full_table['dilution_change_percentage_24h_rank_pct'] = full_table['dilution_change_percentage_24h'].rank(pct=True)
try:
  full_table['token_rotation_24h_rank_pct'] = full_table['token_rotation_24h'].rank(pct=True)
except Exception:
  pass

full_table['circulating_cap_rank_pct'] = full_table['circulating_cap'].rank(pct=True)
full_table['total_cap_rank_pct'] = full_table['total_cap'].rank(pct=True)
full_table['max_cap_rank_pct'] = full_table['max_cap'].rank(pct=True)
full_table['price_change_percentage_24h_rank_pct'] = full_table['price_change_percentage_24h'].rank(pct=True)
full_table['market_cap_change_percentage_24h_rank_pct'] = full_table['market_cap_change_percentage_24h'].rank(pct=True)
del full_table['market_cap_rank']
full_table['market_cap_rank_pct'] = full_table['market_cap'].rank(pct=True)

full_table['dilution_ratio'] = (full_table['max_supply'].values / full_table['circulating_supply'].values) -1

full_table.replace([np.inf, -np.inf], np.nan, inplace=True)
full_table.info()

  0%|          | 0/135 [00:00<?, ?it/s]

In [16]:
#@title Load Prices to SQLite { display-mode: "code" }

proprietary_screener = "proprietary_screener"

full_table.to_sql(proprietary_screener, conn_SQLite, if_exists='replace', index=False) # writes to file
conn_SQLite.commit()

# Just be sure any changes have been committed or they will be lost.

proprietary_screener = pd.read_sql('select * from {}'.format(proprietary_screener), conn_SQLite)

# Futures Coverage Indicator

In [11]:
#Query the derivatives table

# Group table By index ID and create a: where if the  contract type includes perpetual, at P Contract type Futures exist, at letter



In [12]:
proprietary_screener.head()

,id,symbol,name,image,current_price,market_cap,fully_diluted_valuation,total_volume,high_24h,low_24h,...,CoinGecko_link,dilution_change_percentage_24h_rank_pct,token_rotation_24h_rank_pct,circulating_cap_rank_pct,total_cap_rank_pct,max_cap_rank_pct,price_change_percentage_24h_rank_pct,market_cap_change_percentage_24h_rank_pct,market_cap_rank_pct,dilution_ratio
0,bitcoin,BTC,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,20933.000,399367233600,4.393784e+11,1.927862e+10,21151.000,20254.000000,...,https://www.coingecko.com/en/coins/bitcoin,0.353,0.526694,0.999,0.998950,0.998214,0.573,0.531,0.999,0.100186
1,ethereum,ETH,Ethereum,https://assets.coingecko.com/coins/images/279/...,1223.790,146237238197,NaN,1.327349e+10,1235.100,1141.500000,...,https://www.coingecko.com/en/coins/ethereum,0.121,0.709446,0.995,0.994748,NaN,0.883,0.875,0.995,NaN
2,tether,USDT,Tether,https://assets.coingecko.com/coins/images/325/...,1.000,66027331894,NaN,4.770971e+10,1.003,0.995699,...,https://www.coingecko.com/en/coins/tether,0.903,0.988706,0.991,0.990546,NaN,0.097,0.157,0.991,NaN
3,usd-coin,USDC,USD Coin,https://assets.coingecko.com/coins/images/6319...,1.002,55514336681,NaN,4.427233e+09,1.005,0.997883,...,https://www.coingecko.com/en/coins/usd-coin,0.249,0.678645,0.987,0.986345,NaN,0.167,0.121,0.987,NaN
4,binancecoin,BNB,BNB,https://assets.coingecko.com/coins/images/825/...,240.370,39234447697,3.967654e+10,9.419452e+08,243.150,235.680000,...,https://www.coingecko.com/en/coins/binancecoin,0.387,0.341889,0.983,0.982143,0.991071,0.263,0.243,0.983,0.011268


In [13]:
proprietary_screener.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 46 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   id                                         500 non-null    object 
 1   symbol                                     500 non-null    object 
 2   name                                       500 non-null    object 
 3   image                                      500 non-null    object 
 4   current_price                              500 non-null    float64
 5   market_cap                                 500 non-null    int64  
 6   fully_diluted_valuation                    280 non-null    float64
 7   total_volume                               500 non-null    float64
 8   high_24h                                   500 non-null    float64
 9   low_24h                                    500 non-null    float64
 10  price_change_24h          

In [14]:
#@title Gets Currencies { display-mode: "code" }
output = cg.get_supported_vs_currencies()
data = pd.DataFrame(output, columns =['currencies'])
CoinGecko_currencies = "CoinGecko_currencies"



data.to_sql(CoinGecko_currencies, conn_SQLite, if_exists='replace', index=False) # writes to file
conn_SQLite.commit()

# Just be sure any changes have been committed or they will be lost.

CoinGecko_currencies = pd.read_sql('select * from {}'.format(CoinGecko_currencies), conn_SQLite)

In [15]:
#@title
output = cg.get_derivatives()
data = pd.json_normalize(output,max_level=4)
data

,market,symbol,index_id,price,price_percentage_change_24h,contract_type,index,basis,spread,funding_rate,open_interest,volume_24h,last_traded_at,expired_at
0,Binance (Futures),BTCUSDT,BTC,20916.19,3.448600,perpetual,20927.159456,0.048092,0.01,0.010000,2.351540e+09,1.226689e+10,1657217534,NaN
1,Bitget Futures,BTCUSDT_UMCBL,BTC,20936.16,3.422983,perpetual,20937.959309,-0.002582,0.01,-0.006800,1.971912e+09,4.914379e+09,1657217519,NaN
2,FTX (Derivatives),BTC-PERP,BTC,20921.0,3.477100,perpetual,20927.705000,0.032049,0.01,-0.014400,1.360255e+09,2.171068e+09,1657217453,NaN
3,Binance (Futures),ETHUSDT,ETH,1223.51,7.487284,perpetual,1224.147171,0.047989,0.01,0.007147,1.229457e+09,7.084106e+09,1657217534,NaN
4,MEXC Global (Futures),BTC_USDT,BTC,20926.16,3.391463,perpetual,20938.100000,0.045870,0.01,0.005000,9.773849e+08,1.475068e+09,1657217515,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3093,FTX (Derivatives),GLD-0930,GLD,162.73,0.323674,futures,162.275000,-0.276540,0.11,0.000000,NaN,4.874250e+00,1657208479,1.664568e+09
3094,FTX (Derivatives),CGC-0930,CGC,2.68,0.000000,futures,2.750000,2.803738,0.81,0.000000,1.393434e+04,8.290580e+02,1657135073,1.664568e+09
3095,FTX (Derivatives),BNTX-0930,BNTX,159.77,-0.646726,futures,167.230000,4.669212,0.16,0.000000,5.176550e+03,7.127050e+02,1657140786,1.664568e+09
3096,FTX (Derivatives),GDXJ-0930,GDXJ,30.33,0.000000,futures,31.280000,3.132212,0.19,0.000000,NaN,1.213200e+03,1657156712,1.664568e+09


In [ ]:
#@title Get Global Data { display-mode: "code" }

output = cg.get_global()
data = pd.json_normalize(output,max_level=4)
data = pd.DataFrame(output)

CoinGecko_global = "CoinGecko_global"

data.to_sql(CoinGecko_global, conn_SQLite, if_exists='replace', index=False) # writes to file
conn_SQLite.commit()

# Just be sure any changes have been committed or they will be lost.

CoinGecko_global = pd.read_sql('select * from {}'.format(CoinGecko_global), conn_SQLite)
CoinGecko_global.info()

In [ ]:
#@title Get Assess platforms { display-mode: "code" }

output = cg.get_asset_platforms()

data = pd.json_normalize(output,max_level=4)
data = pd.DataFrame(output)



CoinGecko_asset_platforms = "CoinGecko_asset_platforms"

data.to_sql(CoinGecko_asset_platforms, conn_SQLite, if_exists='replace', index=False) # writes to file
conn_SQLite.commit()

# Just be sure any changes have been committed or they will be lost.

CoinGecko_asset_platforms = pd.read_sql('select * from {}'.format(CoinGecko_asset_platforms), conn_SQLite)
CoinGecko_asset_platforms.info()

**bold text**# Create Features

In [ ]:
#@title
# Just be sure any changes have been committed or they will be lost.
conn_SQLite.close()

# Backtesting

# Chartbook
